In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
import copy
import os
import glob
import pylab as pl
import numpy as np
import matplotlib.animation as animation
import types
from mpl_toolkits.basemap import cm,addcyclic,Basemap, shiftgrid
from ArcticTools import map
#import seaborn as sns

% matplotlib inline

In [ ]:
nc_filename=sorted(glob.glob\
        ('/ocean/xiaoxiny/research/result_jasper/data_exh005_btscav_v2/EXH005-btscav_v2_19912016.nc'))#*_ptrc_T*.nc'))

print("found *.nc file: \n{}".format(nc_filename))
ptrc=nc.Dataset(nc_filename[0])

lon=ptrc.variables['nav_lon'][:,:]
lat=ptrc.variables['nav_lat'][:,:]
dep = ptrc.variables['deptht'][:]


tmask=nc.Dataset(
        '/ocean/xiaoxiny/research/NEMO-code/NEMOGCM/CONFIG/myANHA/EXP00/crop.nc')
mbathy=tmask.variables['mbathy'][0,400:,:]
tmask=tmask.variables['tmask'][0,:,400:,:]


bathy=nc.Dataset(
    '/ocean/xiaoxiny/research/data/Jasper/ANHA4_bathy_etopo1_gebco1_smoothed_coast_corrected_mar10.nc')
bathy=bathy.variables['Bathymetry'][400:,:]

In [ ]:
# snapshot

In [ ]:
# sns.set_style('whitegrid')

In [ ]:
from pylab import cm

In [ ]:
def plot(years,layer= 30,ptrc=nc.Dataset(nc_filename[0]),string='TR_8',title ='Dissolved $^{230}$Th',cont_label = 'F'):
    fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(17/1.5,15/1.5))
    for ax,time,index,phase in zip(axes.flat,np.array(years)-1991,
                                   ['a','b','c','d','e','f'],['I','II','III','IV','V','VI','VII']):
        data =ptrc.variables[string][time,:,:,:]
        data =np.ma.masked_where(tmask==0,data)
        #data[:,:,:]=np.ma.masked

        m = Basemap(width =3e6,height =2.5e6,lon_0=-140, lat_0=79,projection='stere', resolution='i', ax=ax)
        m.drawcoastlines()
        m.fillcontinents(color = '0.5')
        if  index == 'a'   or index == 'c' :                                           #l,r,top,bot
            m.drawparallels([70],labels=[1, 0, 0, 1], fontsize=13.5)
        else:
            m.drawparallels([70],labels=[0, 1, 0, 0], fontsize=13.5)
        if index == 'c' or index =='d':
            m.drawmeridians([-140],labels=[0, 1, 0, 1], fontsize=13.5)
        else:
            m.drawmeridians([-140],labels=[0, 0, 0, 0], fontsize=13.5)
        x_lon,y_lat = m(lon,lat)
        # pcolor + contour
        if index == 'a':
            data0_contour = data
        levels = [0.1,0.2,0.3,0.4,0.5]
        #CS1=m.contour(x_lon,y_lat,data0_contour[layer,:,:],levels,ls='.',colors = 'gray',linewidths=1);# cmap=cm.gray_r
        
        if cont_label != 'F':
            #plt.clabel(CS1, inline=1, fmt = '%.1f', fontsize=13.5)
            CS2=m.contour(x_lon,y_lat,data[layer,:,:],levels,cmap=cm.GnBu,linewidths=1); #colors='k'
            plt.clabel(CS2, inline=1, fmt = '%.1f', fontsize=13.5)
        cs=m.pcolor(x_lon,y_lat,data[layer,:,:],cmap='GnBu',vmin=0.1,vmax=0.4); #GnBu, YlGnBu RdYlBu_r

        ax.text(0.01,0.95, '(%s) Year: %s'%(index, time+1991), transform=ax.transAxes, ha='left', fontsize=13.5)
        # bathy
        m.contour(x_lon,y_lat,bathy,[1000,2000,3000],linewidth=0.075,colors='black',alpha=0.1)
        for aug in (['bottom','left','right','top']):
            ax.spines[aug].set_color('k') 
        
        for i in range (len(data_Station)):
            data_x_lon,data_y_lat = m(data_lon,data_lat)
            if np.int(data_Station[i][0][:4]) == time+1991: # sample year == model year
                
                m.scatter(data_x_lon[i],data_y_lat[i],s=data_Th[i][0]*500,\
                      c = data_Th[i][0],cmap=cm.GnBu,vmin=0.1,vmax=0.4,label = data_Station[i][0])

        #plt.legend(loc=4,prop={'weight':'bold'},bbox_to_anchor=(0.28, 0.26),ncol=1,frameon=False)
    
    
    


    cax = fig.add_axes([0.22, -0.00, 0.6, 0.02])
    cbar=fig.colorbar(cs,cax=cax,extend='both',orientation='horizontal')
    cbar.ax.tick_params(labelsize=12.5) 
    cbar.set_label('%s conc. (dpm/m$^3$) at intermediate depth (500 m)'%(title),fontsize=13.5)
    left  = 0.05  # the left side of the subplots of the figure
    right = 0.95    # the right side of the subplots of the figure
    bottom = 0.05   # the bottom of the subplots of the figure
    top = 0.95      # the top of the subplots of the figure
    wspace = 0.1/2   # the amount of width reserved for blank space between subplots
    hspace = 0.0   # the amount of height reserved for white space between subplots
    plt.subplots_adjust(left, bottom, right, top, wspace, hspace)
    

In [ ]:
read_dictionary = np.load('Thdata.npy').item()

In [ ]:
data_lon = read_dictionary['x_lon']
data_lat = read_dictionary['y_lat']
data_Th = read_dictionary['Th']
data_Station = read_dictionary['Station']

In [ ]:
plot(years = [2002,2007,2009,2015],cont_label = 'F')
plt.savefig('/ocean/xiaoxiny/research/analysis-cindy/proj2_notebooks/Untitled Folder/fig4.pdf')

In [ ]:
data_Th[9][0]